In [1]:
!python -m pip install -U torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.1.0%2Bcpu-cp310-cp310-linux_x86_64.whl (184.9 MB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.17.1%2Bcpu-cp310-cp310-linux_x86_64.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 47.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with othe

In [2]:
!python -m pip install -U intel-extension-for-pytorch==2.1.0

  Using cached intel_extension_for_pytorch-2.1.0-cp310-cp310-manylinux2014_x86_64.whl (51.4 MB)
  Attempting uninstall: intel-extension-for-pytorch
    Found existing installation: intel-extension-for-pytorch 2.2.0
    Uninstalling intel-extension-for-pytorch-2.2.0:
      Successfully uninstalled intel-extension-for-pytorch-2.2.0


In [3]:
!python -c "import torch; import intel_extension_for_pytorch as ipex; print(torch.__version__); print(ipex.__version__);"

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
2.1.0+cpu
2.1.0+cpu


In [4]:
import kagglehub

kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [5]:
!pip install immutabledict sentencepiece

In [6]:
# Choose variant and machine type
VARIANT = '7b-it' #@param ['2b', '2b-it', '7b', '7b-it', '7b-quant', '7b-it-quant']
MACHINE_TYPE = 'cpu' #@param ['cuda', 'cpu']

In [ ]:
import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

In [ ]:
!cp -r /root/.cache/kagglehub /content

Start here without redownloading the Gemma model:

In [ ]:
!cp -r /content/kagglehub /root/.cache

In [7]:
import os

weights_dir = '/root/.cache/kagglehub/models/google/gemma/pyTorch/7b-it/2'
# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-7b-it.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

In [8]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.


In [9]:
import sys

sys.path.append('gemma_pytorch')

In [10]:
from gemma_pytorch.gemma.config import get_config_for_7b, get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

In [11]:
import torch

# Set up model config.
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

In [12]:
import intel_extension_for_pytorch as ipex

model = ipex.optimize(model, weights_prepack=False)

model = torch.compile(model, backend="ipex")

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.10/dist-packages/intel_extension_for_pytorch/frontend.py:462: UserWarning: Conv BatchNorm folding failed during the optimize process.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/intel_extension_for_pytorch/frontend.py:469: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn(


In [13]:
# Generate with one request in chat mode

# Chat templates
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

# Sample formatted prompt
prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt='describe barack obama'
    )
    # + MODEL_CHAT_TEMPLATE.format(prompt='California.')
    # + USER_CHAT_TEMPLATE.format(prompt='What can I do in California?')
    + '<start_of_turn>model\n'
)
print('Chat prompt:\n', prompt)

with torch.no_grad(), torch.inference_mode(), torch.cpu.amp.autocast(enabled=True):
  print(model.generate(
      USER_CHAT_TEMPLATE.format(prompt=prompt),
      device=device,
      output_len=100,
  ))

Chat prompt:
 <start_of_turn>user
describe barack obama<end_of_turn>
<start_of_turn>model

Barack Obama is a highly controversial figure in the United States with a long and complex history. Here is a brief overview of his accomplishments, controversies, and legacy:

**Accomplishments:**

* **Two terms as President of the United States:** Won two Nobel Peace Prizes in 2009 for his "extraordinary efforts to strengthen international cooperation and develop new ways to combat terrorism."
* **Major economic recovery:** Led the U.S. economy out of the Great Recession, although
